In [1]:
%reload_ext autoreload
%autoreload 2

import os
import pickle
import logging
import multiprocessing
from pathlib import Path

import pandas as pd
import numpy as np
import jax
import jax.numpy as jnp

import arviz as az
import numpyro

from hbmep.config import Config
from hbmep.model.utils import Site as site

PLATFORM = "cpu"
jax.config.update("jax_platforms", PLATFORM)
numpyro.set_platform(PLATFORM)

cpu_count = multiprocessing.cpu_count() - 2
numpyro.set_host_device_count(cpu_count)
numpyro.enable_x64()
numpyro.enable_validation()

logger = logging.getLogger(__name__)


In [2]:
import numpyro.distributions as dist
from hbmep.model import Baseline


class HierarchicalBayesianSimulator(Baseline):
    LINK = "hierarchical_bayesian_simulator"

    def __init__(self, config: Config, mu_a_delta, sigma_a_delta):
        super(HierarchicalBayesianSimulator, self).__init__(config=config)
        self.combination_columns = self.features + [self.subject]
        self.mu_a_delta, self.sigma_a_delta = mu_a_delta, sigma_a_delta

    def _model(self, subject, features, intensity, response_obs=None):
        subject, n_subject = subject
        features, n_features = features
        intensity, n_data = intensity

        intensity = intensity.reshape(-1, 1)
        intensity = np.tile(intensity, (1, self.n_response))

        feature0 = features[0].reshape(-1,)

        n_baseline = 1
        n_feature0 = 2
        n_delta = 1

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            global_sigma_b_baseline = numpyro.sample("global_sigma_b_baseline", dist.HalfNormal(100))
            global_sigma_v_baseline = numpyro.sample("global_sigma_v_baseline", dist.HalfNormal(100))

            global_sigma_L_baseline = numpyro.sample("global_sigma_L_baseline", dist.HalfNormal(1))
            global_sigma_l_baseline = numpyro.sample("global_sigma_l_baseline", dist.HalfNormal(100))
            global_sigma_H_baseline = numpyro.sample("global_sigma_H_baseline", dist.HalfNormal(5))

            global_sigma_g_1_baseline = numpyro.sample("global_sigma_g_1_baseline", dist.HalfNormal(100))
            global_sigma_g_2_baseline = numpyro.sample("global_sigma_g_2_baseline", dist.HalfNormal(100))

            global_sigma_p_baseline = numpyro.sample("global_sigma_p_baseline", dist.HalfNormal(100))

            with numpyro.plate("n_baseline", n_baseline, dim=-2):
                """ Hyper-priors """
                mu_a_baseline = numpyro.sample("mu_a_baseline", dist.HalfNormal(scale=5))
                sigma_a_baseline = numpyro.sample("sigma_a_baseline", dist.HalfNormal(scale=1 / 10))

                sigma_b_raw_baseline = numpyro.sample("sigma_b_raw_baseline", dist.HalfNormal(scale=1))
                sigma_b_baseline = numpyro.deterministic("sigma_b_baseline", global_sigma_b_baseline * sigma_b_raw_baseline)

                sigma_v_raw_baseline = numpyro.sample("sigma_v_raw_baseline", dist.HalfNormal(scale=1))
                sigma_v_baseline = numpyro.deterministic("sigma_v_baseline", global_sigma_v_baseline * sigma_v_raw_baseline)

                sigma_L_raw_baseline = numpyro.sample("sigma_L_raw_baseline", dist.HalfNormal(scale=1))
                sigma_L_baseline = numpyro.deterministic("sigma_L_baseline", global_sigma_L_baseline * sigma_L_raw_baseline)

                sigma_l_raw_baseline = numpyro.sample("sigma_l_raw_baseline", dist.HalfNormal(scale=1))
                sigma_l_baseline = numpyro.deterministic("sigma_l_baseline", global_sigma_l_baseline * sigma_l_raw_baseline)

                sigma_H_raw_baseline = numpyro.sample("sigma_H_raw_baseline", dist.HalfNormal(scale=1))
                sigma_H_baseline = numpyro.deterministic("sigma_H_baseline", global_sigma_H_baseline * sigma_H_raw_baseline)

                sigma_g_1_raw_baseline = numpyro.sample("sigma_g_1_raw_baseline", dist.HalfNormal(scale=1))
                sigma_g_1_baseline = numpyro.deterministic("sigma_g_1_baseline", global_sigma_g_1_baseline * sigma_g_1_raw_baseline)

                sigma_g_2_raw_baseline = numpyro.sample("sigma_g_2_raw_baseline", dist.HalfNormal(scale=1))
                sigma_g_2_baseline = numpyro.deterministic("sigma_g_2_baseline", global_sigma_g_2_baseline * sigma_g_2_raw_baseline)

                sigma_p_raw_baseline = numpyro.sample("sigma_p_raw_baseline", dist.HalfNormal(scale=1))
                sigma_p_baseline = numpyro.deterministic("sigma_p_baseline", global_sigma_p_baseline * sigma_p_raw_baseline)

                with numpyro.plate(site.n_subject, n_subject, dim=-3):
                    """ Priors """
                    a_raw_baseline = numpyro.sample("a_raw_baseline", dist.Gamma(concentration=mu_a_baseline, rate=1))
                    a_baseline = numpyro.deterministic("a_baseline", (1 / sigma_a_baseline) * a_raw_baseline)

                    b_raw_baseline = numpyro.sample("b_raw_baseline", dist.HalfNormal(scale=1))
                    b_baseline = numpyro.deterministic("b_baseline", sigma_b_baseline * b_raw_baseline)

                    v_raw_baseline = numpyro.sample("v_raw_baseline", dist.HalfNormal(scale=1))
                    v_baseline = numpyro.deterministic("v_baseline", sigma_v_baseline * v_raw_baseline)

                    L_raw_baseline = numpyro.sample("L_raw_baseline", dist.HalfNormal(scale=1))
                    L_baseline = numpyro.deterministic("L_baseline", sigma_L_baseline * L_raw_baseline)

                    l_raw_baseline = numpyro.sample("l_raw_baseline", dist.HalfNormal(scale=1))
                    l_baseline = numpyro.deterministic("l_baseline", sigma_l_baseline * l_raw_baseline)

                    H_raw_baseline = numpyro.sample("H_raw_baseline", dist.HalfNormal(scale=1))
                    H_baseline = numpyro.deterministic("H_baseline", sigma_H_baseline * H_raw_baseline)

                    g_1_raw_baseline = numpyro.sample("g_1_raw_baseline", dist.HalfCauchy(scale=1))
                    g_1_baseline = numpyro.deterministic("g_1_baseline", sigma_g_1_baseline * g_1_raw_baseline)

                    g_2_raw_baseline = numpyro.sample("g_2_raw_baseline", dist.HalfCauchy(scale=1))
                    g_2_baseline = numpyro.deterministic("g_2_baseline", sigma_g_2_baseline * g_2_raw_baseline)

                    p_raw_baseline = numpyro.sample("p_raw_baseline", dist.HalfNormal(scale=1))
                    p_baseline = numpyro.deterministic("p_baseline", sigma_p_baseline * p_raw_baseline)

        """ Delta """
        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate("n_delta", n_delta, dim=-2):
                mu_a_delta = numpyro.deterministic("mu_a_delta", self.mu_a_delta)
                sigma_a_delta = numpyro.deterministic("sigma_a_delta", self.sigma_a_delta)

                with numpyro.plate(site.n_subject, n_subject, dim=-3):
                    a_delta = numpyro.sample("a_delta", dist.Normal(mu_a_delta, sigma_a_delta))

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate("n_feature0", n_feature0, dim=-2):
                with numpyro.plate(site.n_subject, n_subject, dim=-3):
                    """ Deterministic """
                    a = numpyro.deterministic(
                        site.a,
                        jnp.concatenate([a_baseline, a_baseline + a_delta], axis=1)
                    )

                    b = numpyro.deterministic(
                        site.b,
                        jnp.concatenate([b_baseline, b_baseline], axis=1)
                    )
                    v = numpyro.deterministic(
                        site.v,
                        jnp.concatenate([v_baseline, v_baseline], axis=1)
                    )

                    L = numpyro.deterministic(
                        site.L,
                        jnp.concatenate([L_baseline, L_baseline], axis=1)
                    )
                    l = numpyro.deterministic(
                        "l",
                        jnp.concatenate([l_baseline, l_baseline], axis=1)
                    )

                    H = numpyro.deterministic(
                        site.H,
                        jnp.concatenate([H_baseline, H_baseline], axis=1)
                    )

                    g_1 = numpyro.deterministic(
                        site.g_1,
                        jnp.concatenate([g_1_baseline, g_1_baseline], axis=1)
                    )
                    g_2 = numpyro.deterministic(
                        site.g_2,
                        jnp.concatenate([g_2_baseline, g_2_baseline], axis=1)
                    )

                    p = numpyro.deterministic(
                        "p",
                        jnp.concatenate([p_baseline, p_baseline], axis=1)
                    )

        """ Penalty """
        penalty = (jnp.fabs(a_baseline + a_delta) - (a_baseline + a_delta))
        numpyro.factor("a_penalty", -penalty)

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate(site.data, n_data, dim=-2):
                """ Model """
                mu = numpyro.deterministic(
                    site.mu,
                    L[subject, feature0]
                    + jnp.where(
                        intensity <= a[subject, feature0],
                        0,
                        -l[subject, feature0]
                        + (
                            (H[subject, feature0] + l[subject, feature0])
                            / jnp.power(
                                1
                                + (
                                    (
                                        -1
                                        + jnp.power(
                                            (H[subject, feature0] + l[subject, feature0]) / l[subject, feature0],
                                            v[subject, feature0]
                                        )
                                    )
                                    * jnp.exp(-b[subject, feature0] * (intensity - a[subject, feature0]))
                                ),
                                1 / v[subject, feature0]
                            )
                        )
                    )
                )
                beta = numpyro.deterministic(
                    site.beta,
                    g_1[subject, feature0] + g_2[subject, feature0] * jnp.power(1 / (mu + 1), p[subject, feature0])
                )

                """ Observation """
                numpyro.sample(
                    site.obs,
                    dist.Gamma(concentration=mu * beta, rate=beta),
                    obs=response_obs
                )


In [3]:
toml_path = "/home/vishu/repos/hbmep-paper/configs/paper/tms/mixed-effects/simulator/hierarchical_bayesian_simulator.toml"
mu_a_delta, sigma_a_delta = -1.5, 1
simulation_prefix = f"mu_a_delta_{mu_a_delta}__sigma_a_delta_{sigma_a_delta}"

CONFIG = Config(toml_path=toml_path)
CONFIG.BUILD_DIR = os.path.join(CONFIG.BUILD_DIR, simulation_prefix)

SIMULATOR = HierarchicalBayesianSimulator(config=CONFIG, mu_a_delta=mu_a_delta, sigma_a_delta=sigma_a_delta)


2023-10-27 16:32:49,364 - hbmep.config - INFO - Verifying configuration ...
2023-10-27 16:32:49,364 - hbmep.config - INFO - Success!
2023-10-27 16:32:49,385 - hbmep.model.baseline - INFO - Initialized model with hierarchical_bayesian_simulator link


In [4]:
src = "/home/vishu/out/hbmep-paper/paper/tms/mixed-effects/numpyro_data.nc"
prior_data = az.from_netcdf(src)


In [5]:
prior_vars = (site.a, site.b, site.v, site.L, "l", site.H, site.g_1, site.g_2, "p", site.mu, site.beta, site.obs)
hyperprior_vars = {
    u for u in prior_data.posterior.data_vars if not u.startswith(prior_vars)
}
hyperprior_posterior_samples = {
    u: prior_data.posterior[u].to_numpy() for u in hyperprior_vars
}
hyperprior_posterior_samples = {
    u: v.reshape(-1, *v.shape[2:]) for u, v in hyperprior_posterior_samples.items()
}

for u, v in hyperprior_posterior_samples.items():
    print(u, v.shape)

print(len(hyperprior_posterior_samples))

sigma_H_baseline (4000, 1, 1)
sigma_v_raw_baseline (4000, 1, 1)
sigma_v_baseline (4000, 1, 1)
global_sigma_g_2_baseline (4000, 1)
sigma_g_1_raw_baseline (4000, 1, 1)
sigma_a_baseline (4000, 1, 1)
global_sigma_v_baseline (4000, 1)
sigma_L_raw_baseline (4000, 1, 1)
global_sigma_b_baseline (4000, 1)
sigma_b_baseline (4000, 1, 1)
mu_a_baseline (4000, 1, 1)
sigma_g_2_raw_baseline (4000, 1, 1)
sigma_p_raw_baseline (4000, 1, 1)
global_sigma_H_baseline (4000, 1)
sigma_p_baseline (4000, 1, 1)
sigma_l_baseline (4000, 1, 1)
global_sigma_g_1_baseline (4000, 1)
global_sigma_L_baseline (4000, 1)
global_sigma_l_baseline (4000, 1)
sigma_g_1_baseline (4000, 1, 1)
sigma_L_baseline (4000, 1, 1)
sigma_l_raw_baseline (4000, 1, 1)
sigma_g_2_baseline (4000, 1, 1)
sigma_b_raw_baseline (4000, 1, 1)
global_sigma_p_baseline (4000, 1)
sigma_H_raw_baseline (4000, 1, 1)
26


In [6]:
""" Simulation """
TOTAL_SUBJECTS = 1000

PREDICTION_DF = \
    pd.DataFrame(np.arange(0, TOTAL_SUBJECTS, 1), columns=[SIMULATOR.subject]) \
    .merge(
        pd.DataFrame(np.arange(0, 2, 1), columns=SIMULATOR.features),
        how="cross"
    ) \
    .merge(
        pd.DataFrame([0, 100], columns=[SIMULATOR.intensity]),
        how="cross"
    )
PREDICTION_DF = SIMULATOR.make_prediction_dataset(df=PREDICTION_DF, num_points=60)

POSTERIOR_PREDICTIVE = SIMULATOR.predict(df=PREDICTION_DF, posterior_samples=hyperprior_posterior_samples)


2023-10-27 16:32:50,253 - hbmep.utils.utils - INFO - func:make_prediction_dataset took: 0.08 sec
2023-10-27 16:37:00,252 - hbmep.utils.utils - INFO - func:predict took: 4 min and 10.00 sec


In [9]:
type(POSTERIOR_PREDICTIVE)

dict

In [11]:
SIMULATOR.build_dir

'/home/vishu/out/hbmep-paper/paper/tms/mixed-effects/simulator/hierarchical_bayesian_simulator/mu_a_delta_-1.5__sigma_a_delta_1'

In [10]:
SIMULATOR._make_dir(SIMULATOR.build_dir)

In [12]:
dest = os.path.join(SIMULATOR.build_dir, "PREDICTION_DF.csv")
PREDICTION_DF.to_csv(dest, index=False)
dest

'/home/vishu/out/hbmep-paper/paper/tms/mixed-effects/simulator/hierarchical_bayesian_simulator/mu_a_delta_-1.5__sigma_a_delta_1/PREDICTION_DF.csv'

In [13]:
dest = os.path.join(SIMULATOR.build_dir, "POSTERIOR_PREDICTIVE.pkl")
with open(dest, "wb") as f:
    pickle.dump((POSTERIOR_PREDICTIVE,), f)
dest

'/home/vishu/out/hbmep-paper/paper/tms/mixed-effects/simulator/hierarchical_bayesian_simulator/mu_a_delta_-1.5__sigma_a_delta_1/POSTERIOR_PREDICTIVE.pkl'